## ictv-mmseqs2-protein-database

This repository contains instructions to generate a MMSeqs2 protein database with ICTV taxonomy. This database was not benchmarked. For taxonomic assignment of viral genomes you can try [geNomad](https://github.com/apcamargo/genomad).

### Dependencies:

- [`aria2`](https://github.com/aria2/aria2)
- [`ripgrep`](https://github.com/BurntSushi/ripgrep)
- [`csvtk`](https://github.com/shenwei356/csvtk)
- [`seqkit`](https://github.com/shenwei356/seqkit)
- [`taxonkit` (version 0.11.1)](https://github.com/shenwei356/taxonkit/releases/tag/v0.11.1)
- [`taxopy`](https://github.com/apcamargo/taxopy)
- [`mmseqs2`](https://github.com/soedinglab/MMseqs2)


### Instructions
First we create an move to a comfortable working directory


In [1]:
import os
os.makedirs(f"/clusterfs/jgi/scratch/science/metagen/neri/projects/ictv_antonio",exist_ok=True)
os.chdir(f"/clusterfs/jgi/scratch/science/metagen/neri/projects/ictv_antonio")

 Then, we create a conda enviroment for reproducibilty etc, and to install the dependencies

In [ ]:
# %%bash
# # # Create and activate a new conda environment
# mamba create -n ictv_mmseqs2 python=3.9 -y
# mamba activate ictv_mmseqs2

# # # Install conda packages
# mamba install -c bioconda -c conda-forge \
#     aria2 \
#     ripgrep \
#     csvtk \
#     seqkit \
#     taxonkit=0.11.1 \
#     mmseqs2 \
#     polars
#     -y
# # ### taxonkit has to be 0.11.1 !!
# # # Install taxopy using pip
# pip install taxopy ipython jupyter
# # # Note- now change your i/pythonkernel to the one you just created

### Verify installations


In [2]:
%%bash
echo "Checking installed tools:"
aria2c --version | head -n 1
rg --version | head -n 1
csvtk version | head -n 1
seqkit version | head -n 1
taxonkit version | head -n 1
python -c "import taxopy; print(f'taxopy {taxopy.__version__}')"
mmseqs version | head -n 1

echo -e "\nSetup complete. You can now run the rest of the thing."

Checking installed tools:
aria2 version 1.37.0
ripgrep 14.1.0 (rev 9477456963)
csvtk v0.30.0
seqkit v2.8.2
taxonkit v0.11.1
taxopy 0.13.0
15.6f452

Setup complete. You can now run the rest of the thing.


Now, download the latest VMR release from ICTV, and refseq & genbank assembly status files from NCBI's FTP.


In [ ]:
%%bash
aria2c -x 4 -o ictv.xlsx "https://ictv.global/vmr/current?fid=15873" --check-certificate=false
# aria2c -x 4 -o assembly_summary_genbank.tsv "https://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/assembly_summary_genbank.txt" --check-certificate=false
# aria2c -x 4 -o assembly_summary_refseq.tsv "https://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/assembly_summary_refseq.txt" --check-certificate=false


[#cf1710 0B/1.2GiB(0%) CN:1 DL:0B] [FileAlloc:#cf1710 758MiB/1.2GiB(59%)]
[#cf1710 0B/1.2GiB(0%) CN:1 DL:0B]
[#cf1710 14MiB/1.2GiB(1%) CN:4 DL:14MiB ETA:1m26s]
[#cf1710 39MiB/1.2GiB(3%) CN:4 DL:20MiB ETA:1m1s]
[#cf1710 61MiB/1.2GiB(4%) CN:4 DL:20MiB ETA:58s]
[#cf1710 75MiB/1.2GiB(5%) CN:4 DL:18MiB ETA:1m3s]
[#cf1710 92MiB/1.2GiB(7%) CN:4 DL:18MiB ETA:1m3s]
[#cf1710 120MiB/1.2GiB(9%) CN:4 DL:20MiB ETA:56s]
[#cf1710 164MiB/1.2GiB(12%) CN:4 DL:23MiB ETA:46s]
[#cf1710 210MiB/1.2GiB(16%) CN:4 DL:26MiB ETA:39s]
[#cf1710 238MiB/1.2GiB(18%) CN:4 DL:26MiB ETA:38s]
[#cf1710 258MiB/1.2GiB(20%) CN:4 DL:26MiB ETA:38s]
[#cf1710 279MiB/1.2GiB(21%) CN:4 DL:26MiB ETA:37s]
[#cf1710 303MiB/1.2GiB(23%) CN:4 DL:26MiB ETA:36s]
[#cf1710 336MiB/1.2GiB(26%) CN:4 DL:27MiB ETA:33s]
[#cf1710 371MiB/1.2GiB(29%) CN:4 DL:30MiB ETA:30s]
[#cf1710 384MiB/1.2GiB(30%) CN:4 DL:29MiB ETA:30s]
[#cf1710 392MiB/1.2GiB(30%) CN:4 DL:27MiB ETA:32s]
[#cf1710 405MiB/1.2GiB(31%) CN:4 DL:24MiB ETA:35s]
[#cf1710 426MiB/1.2GiB(33%) CN

#### convert ICTV's xlsx to tsv


In [72]:
%%bash
csvtk xlsx2csv ictv.xlsx \
    | csvtk csv2tab \
    | sed 's/\xc2\xa0/ /g' \
    | csvtk replace -t -F -f "*" -p "^\s+|\s+$" \
    > ictv.tsv

# choose columns, and remove duplicates
csvtk cut -t -f "Realm,Subrealm,Kingdom,Subkingdom,Phylum,Subphylum,Class,Subclass,Order,Suborder,Family,Subfamily,Genus,Subgenus,Species" ictv.tsv \
    | csvtk uniq -t -f "Realm,Subrealm,Kingdom,Subkingdom,Phylum,Subphylum,Class,Subclass,Order,Suborder,Family,Subfamily,Genus,Subgenus,Species" \
    | csvtk del-header -t \
    > ictv.taxonomy.tsv

Create a file that will store all the ICTV taxa names:

In [38]:
%%bash
csvtk cut -t -H -f 1,3,5,7,9,11,13,15 ictv.taxonomy.tsv \
    | sed 's/\t/\n/g' \
    | awk '!/^[[:blank:]]*$/' \
    | sort -u \
    > ictv.names.txt

Now let's get NCBI taxdump 

In [ ]:
%%bash
aria2c  -x 4 -o taxdump.tar.gz  ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz
tar -xzf taxdump.tar.gz

Use `taxonkit create-taxdump` to create a custom taxdump for ICTV. Next, execute the `fix_taxdump.py` script, which will make the taxids sequential to make them compatible with MMSeqs2:

In [ ]:
%%bash
# mkdir ~/.taxonkit/
# mv ./*dmp ~/.taxonkit/
# mv gc.prt ~/.taxonkit/
# mv readme* ~/.taxonkit/
taxonkit create-taxdump -K 1 -P 3 -C 5 -O 7 -F 9 -G 11 -S 13 -T 15 \
    --rank-names "realm","kingdom","phylum","class","order","family","genus","species" \
    ictv.taxonomy.tsv --out-dir ictv-taxdump --force


This next block used to be called "fix_taxdump.py"

In [ ]:
from pathlib import Path

taxid_mapping_dict = {}
count = 1
with open(Path("ictv-taxdump").joinpath("names.dmp")) as fin:
    for line in fin:
        taxid = line.split("\t")[0]
        taxid_mapping_dict[taxid] = str(count)
        count += 1

with open(Path("ictv-taxdump").joinpath("names.dmp")) as fin, open(
    Path("ictv-taxdump").joinpath("names.dmp.new"), "w"
) as fout:
    for line in fin:
        line = line.strip().split("\t")
        line[0] = taxid_mapping_dict[line[0]]
        line = "\t".join(line)
        fout.write(f"{line}\n")

with open(Path("ictv-taxdump").joinpath("nodes.dmp")) as fin, open(
    Path("ictv-taxdump").joinpath("nodes.dmp.new"), "w"
) as fout:
    for line in fin:
        line = line.strip().split("\t")
        line[0] = taxid_mapping_dict[line[0]]
        line[2] = taxid_mapping_dict[line[2]]
        line = "\t".join(line)
        fout.write(f"{line}\n")

Path("ictv-taxdump").joinpath("names.dmp").unlink()
Path("ictv-taxdump").joinpath("nodes.dmp").unlink()
Path("ictv-taxdump").joinpath("nodes.dmp.new").rename("ictv-taxdump/nodes.dmp")
Path("ictv-taxdump").joinpath("names.dmp.new").rename("ictv-taxdump/names.dmp")

Download the NCBI taxdump (again?)

In [ ]:
%%bash
# Download the NCBI taxdump
aria2c -x 4 "ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz" --check-certificate=false #already there
mkdir ncbi-taxdump
tar zxfv taxdump.tar.gz -C ncbi-taxdump
rm taxdump.tar.gz

#### Download the protein → taxid association and filter for viruses
 via the `prot.accession2taxid` file filtered to keep only viral proteins:

In [ ]:
%%bash
aria2c -x 4 "https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/accession2taxid/prot.accession2taxid.FULL.gz" --check-certificate=false

[#26582a 0B/18GiB(0%) CN:1 DL:0B] [FileAlloc:#26582a 1.7GiB/18GiB(9%)]
[#26582a 0B/18GiB(0%) CN:1 DL:0B] [FileAlloc:#26582a 3.9GiB/18GiB(21%)]
[#26582a 0B/18GiB(0%) CN:1 DL:0B] [FileAlloc:#26582a 6.0GiB/18GiB(32%)]
[#26582a 0B/18GiB(0%) CN:1 DL:0B] [FileAlloc:#26582a 8.1GiB/18GiB(43%)]
[#26582a 0B/18GiB(0%) CN:1 DL:0B] [FileAlloc:#26582a 10GiB/18GiB(54%)]
[#26582a 0B/18GiB(0%) CN:1 DL:0B] [FileAlloc:#26582a 12GiB/18GiB(65%)]
[#26582a 0B/18GiB(0%) CN:1 DL:0B] [FileAlloc:#26582a 14GiB/18GiB(77%)]
[#26582a 0B/18GiB(0%) CN:1 DL:0B] [FileAlloc:#26582a 16GiB/18GiB(89%)]
[#26582a 0B/18GiB(0%) CN:1 DL:0B] [FileAlloc:#26582a 18GiB/18GiB(99%)]
[#26582a 0B/18GiB(0%) CN:1 DL:0B]
[#26582a 16MiB/18GiB(0%) CN:4 DL:16MiB ETA:19m49s]
[#26582a 39MiB/18GiB(0%) CN:4 DL:19MiB ETA:16m11s]
[#26582a 66MiB/18GiB(0%) CN:4 DL:22MiB ETA:14m20s]
[#26582a 97MiB/18GiB(0%) CN:4 DL:24MiB ETA:12m56s]
[#26582a 139MiB/18GiB(0%) CN:4 DL:28MiB ETA:11m18s]
[#26582a 195MiB/18GiB(1%) CN:4 DL:32MiB ETA:9m40s]
[#26582a 235MiB/1

In [5]:
import polars as pl
import requests
from concurrent.futures import ThreadPoolExecutor
import os

# Function to download a specific file from the NCBI's assembly FTP directory
def download_protein_file(row):
    url = row['ftp_path']
    taxid = str(row['taxid'])
    file_name = os.path.basename(url.rstrip('/')) + '_protein.faa.gz'
    download_url = os.path.join(url, file_name)

    dest_folder = os.path.join('proteins', taxid)
    local_filename = os.path.join(dest_folder, file_name)

    os.makedirs(dest_folder, exist_ok=True)

    try:
        with requests.get(download_url, stream=True) as r:
            r.raise_for_status()
            with open(local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"Downloaded {download_url} to {local_filename}")
    except Exception as e:
        print(f"Failed to download {download_url}: {e}")

 Read the TSV files, filter to only viruses

In [6]:
refseq_df = pl.read_csv("assembly_summary_refseq.tsv", separator='\t',comment_prefix="##",infer_schema_length=100000,has_header=True)
genbank_df = pl.read_csv("assembly_summary_genbank.tsv", separator='\t',comment_prefix="##",infer_schema_length=100000,has_header=True)

# Concatenate the DataFrames
df = pl.concat([refseq_df, genbank_df])

# Filter for rows where the "group" column is "viral"
df_viral = df.filter(pl.col('group') == 'viral')
df_viral["group"].value_counts()


group,count
str,u32
"""viral""",205904


Use ThreadPoolExecutor to download with n threads at a time

In [ ]:
n_threads = 5  # Adjust as needed, more is gooder?
with ThreadPoolExecutor(max_workers=n_threads) as executor:
    executor.map(download_protein_file, df_viral.to_dicts())

Decompress and concatenate all proteins and create accession2taxid

In [39]:
import os
import glob
import gzip

# Function to concatenate all protein FASTA files into a single file
def concatenate_fasta_files(proteins_dir, output_fasta):
    with open(output_fasta, 'w') as outfile:
        for file_path in glob.glob(os.path.join(proteins_dir, '**/*.faa.gz'), recursive=True):
            with gzip.open(file_path, 'rt') as infile:  # Decompress on the fly
                outfile.write(infile.read())
    print(f"Concatenated all proteins into {output_fasta}")

# Function to create the virus.accession2taxid file - seperated from above as it might change
def create_accession2taxid(proteins_dir, output_tsv):
    with open(output_tsv, 'w') as outfile:
        for file_path in glob.glob(os.path.join(proteins_dir, '**/*.faa.gz'), recursive=True):
            taxid = os.path.basename(os.path.dirname(file_path))
            with gzip.open(file_path, 'rt') as infile:  # Decompress on the fly
                for line in infile:
                    if line.startswith('>'):  # Find FASTA header lines
                        accession = line.split()[0][1:]  # Extract accession (remove '>' and get until first space) - not sure if the defline/comment is to be expected too.
                        outfile.write(f"{accession}\t{taxid}\n")
    print(f"Created virus.accession2taxid file at {output_tsv}")

 Define the directory containing the protein files and the output files


In [40]:
proteins_dir = './proteins'
output_fasta = 'concatenated_proteins.fasta'
output_tsv = 'virus.accession2taxid.tsv'

# Concatenate all protein FASTA files into a single FASTA file
concatenate_fasta_files(proteins_dir, output_fasta)

# Create the virus.accession2taxid TSV file
create_accession2taxid(proteins_dir, output_tsv) # from the NCBI data!


Concatenated all proteins into concatenated_proteins.fasta
Created virus.accession2taxid file at virus.accession2taxid.tsv


In [ ]:
# %%bash
# gunzip prot.accession2taxid.FULL.gz

# awk '{print $2}' prot.accession2taxid.FULL \
#     | sort -u \
#     | taxonkit --data-dir ncbi-taxdump lineage \
#     | rg "\tViruses;" \
#     | awk '{print $1}' \
#     > virus_taxid.list
# csvtk grep --quiet -t -f 2 -P virus_taxid.list prot.accession2taxid.FULL > virus.accession2taxid
# # rm prot.accession2taxid.FULL # why antonio why I thought it crashed and then it deleted it now I need to redownload this wasteful legacy filE??

#### Find the ICTV-compliant proteins and write a new table with the ICTV taxids
Execute the `get_ictv_taxids.py` (moved into the code block below) script to create a `accession2taxid` file with ICTV taxids.

In [7]:
import taxopy

# Read the official ICTV names
with open("ictv.names.txt") as fin:
    ictv_name_set = {i.strip() for i in fin.readlines()}

# Create the Taxopy NCBI and ICTV databases
ncbi_taxdb = taxopy.TaxDb(
    nodes_dmp="ncbi-taxdump/nodes.dmp",
    names_dmp="ncbi-taxdump/names.dmp",
    merged_dmp="ncbi-taxdump/merged.dmp",
    keep_files=True,
)
ictv_taxdb = taxopy.TaxDb(
    nodes_dmp="ictv-taxdump/nodes.dmp",
    names_dmp="ictv-taxdump/names.dmp",
    keep_files=True,
)


Replace non-ICTV taxids


In [ ]:
with open("virus.accession2taxid.tsv") as fin:
 with open("virus.accession2taxid.ictv", "w") as fout:
    for line in fin:
        acc, taxid = line.split("\t")
        ncbi_taxon = taxopy.Taxon(int(taxid), ncbi_taxdb)
        for j in ncbi_taxon.name_lineage:
            if j in ictv_name_set:
                ictv_taxid = taxopy.taxid_from_name(j, ictv_taxdb)
                ictv_taxon = taxopy.Taxon(ictv_taxid[0], ictv_taxdb)
                fout.write(f"{acc}\t{ictv_taxon.taxid}\n")
                # break

Concatenate the virus proteins associated with ICTV viruses:

In [3]:
%%bash
# # Create a list containing the accessions of the proteins of ICTV viruses
cut -f 1 virus.accession2taxid.ictv > virus.accession.txt

# # Filter the NR proteins to keep the proteins encoded by ICTV viruses
# seqkit grep -j 4 -f virus.accession.txt nr.gz | seqkit seq -i -w 0 -o nr.virus.faa.gz
seqkit grep -j 4 -f virus.accession.txt concatenated_proteins.fasta | seqkit seq -i -w 0 -o nr.virus.faa.gz

There will be proteins in `virus.accession2taxid.ictv` that are not in NR. So we will keep only the proteins that are present in the filtered NR FASTA file:

In [4]:
%%bash
# Filter the NR virus taxid table
seqkit fx2tab -n -i nr.virus.faa.gz > nr.virus.list.txt
csvtk grep -t -H -f 1 -P nr.virus.list.txt virus.accession2taxid.ictv > nr.virus.accession2taxid.ictv

[INFO] processed records: 8192
[INFO] processed records: 16384
[INFO] processed records: 24576
[INFO] processed records: 32768
[INFO] processed records: 40960
[INFO] processed records: 49152
[INFO] processed records: 57344
[INFO] processed records: 65536
[INFO] processed records: 73728
[INFO] processed records: 81920
[INFO] processed records: 90112
[INFO] processed records: 98304
[INFO] processed records: 106496
[INFO] processed records: 114688
[INFO] processed records: 122880
[INFO] processed records: 131072
[INFO] processed records: 139264
[INFO] processed records: 147456
[INFO] processed records: 155648
[INFO] processed records: 163840
[INFO] processed records: 172032
[INFO] processed records: 180224
[INFO] processed records: 188416
[INFO] processed records: 196608
[INFO] processed records: 204800
[INFO] processed records: 212992
[INFO] processed records: 221184
[INFO] processed records: 229376
[INFO] processed records: 237568
[INFO] processed records: 245760
[INFO] processed record

Using the filtered FASTA, the ICTV taxdump, and the `virus.accession2taxid.ictv` tabular file, we will create a MMSeqs2 protein database with taxonomy information:

In [5]:
%%bash
# Create the MMSeqs2 database
mkdir virus_tax_db
mmseqs createdb --dbtype 1 nr.virus.faa.gz virus_tax_db/virus_tax_db
mmseqs createtaxdb virus_tax_db/virus_tax_db tmp --ncbi-tax-dump ictv-taxdump --tax-mapping-file nr.virus.accession2taxid.ictv
rm -rf tmp

createdb --dbtype 1 nr.virus.faa.gz virus_tax_db/virus_tax_db 

MMseqs Version:       	15.6f452
Database type         	1
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[===================================================================================================	1 Mio. sequences processed
===================================================================================================	2 Mio. sequences processed
===================================================================================================	3 Mio. sequences processed
===================================================================================================	4 Mio. sequences processed
===================================================================================================	5 Mio. sequences processed
Time for merging to virus_tax_db_h: 0h 0m 1s 292ms
Time for merging 

Finally, to assign taxonomy to sequences in the ICTV """chellenge""" (quatation as I hope it would make the powers to be realize half of the "higher level taxa" are related by hand waving, which is fun we all like to be a little philosophical, I guess...)

In [2]:
%%bash
# wget https://github.com/ICTV-VBEG/ICTV-TaxonomyChallenge/raw/main/dataset/dataset_challenge.tar.gz?download= -O dataset_challenge.tar.gz -o /dev/null
tar -xzf dataset_challenge.tar.gz # considering all of these are non segmented, i.e. 1 contig per input fasta file - this is a really annoying way to bundle them up (why not 1 multifasta file?)
seqkit stats dataset_challenge/*fasta > stats.info

Now we'll concatenate the many single fasta files, and use mmseqs taxonomy on that.

In [4]:
%%bash
cat dataset_challenge/*fasta > ictv_chellenge.fasta
mkdir ictv_chellenge taxonomy_results/
mmseqs createdb ictv_chellenge.fasta ictv_chellenge/mmdb

mmseqs taxonomy ictv_chellenge/mmdb virus_tax_db/virus_tax_db taxonomy_results/resdb2 tmp -e 0.001 -s 7 --blacklist "" --tax-lineage 1 --threads 14 --lca-ranks superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,class,subclass,order,suborder,family,subfamily,genus,subgenus,species

taxonomy ictv_chellenge/mmdb virus_tax_db/virus_tax_db taxonomy_results/resdb2 tmp -e 1e-5 -s 6 --blacklist  --tax-lineage 1 --threads 14 --lca-ranks superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,class,subclass,order,suborder,family,subfamily,genus,subgenus,species 

MMseqs Version:                        	15.6f452
ORF filter                             	1
ORF filter e-value                     	100
ORF filter sensitivity                 	2
LCA mode                               	3
Taxonomy output mode                   	0
Majority threshold                     	0.5
Vote mode                              	1
LCA ranks                              	superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,class,subclass,order,suborder,family,subfamily,genus,subgenus,species
Column with taxonomic lineage          	1
Compressed                             	0
Threads                                	14
Verbosity                              	3
Taxon blacklist               

In [ ]:
%%bash
mmseqs createtsv ictv_chellenge/mmdb taxonomy_results/resdb2 taxonomyResult.tsv

Next we'll read the results and format them as the ICTV chellenge wants

In [9]:
import polars as pl
taxonomyResult = pl.read_csv('taxonomyResult.tsv', separator='\t', has_header=False,new_columns="qseqid,LCA_ncbi_taxid,rank,taxon_name,protein_fragments,protein_fragments_retained,protein_fragments_assigned,protein_fragments_in_agreement_with_lca,lineage".split(',')) 
# Rename columns for clarity
taxonomyResult_mini = taxonomyResult.rename(
    {'qseqid':"SequenceID",'LCA_ncbi_taxid' : 'tax_id',"taxon_name" : "taxname","protein_fragments_in_agreement_with_lca" : "score"}).select(pl.col(["SequenceID","rank","tax_id","taxname","score","lineage"]))

In [10]:
def drop_null_columns(df):
    return df.drop([col for col in df.columns if df[col].is_null().all()])

def capitalize_column_names(df: pl.DataFrame) -> pl.DataFrame:
    # Create a dictionary mapping old column names to capitalized names
    new_names = {col: col.capitalize() for col in df.columns}
    
    # Rename the columns using the dictionary
    return df.rename(new_names)

def order_columns_alphabetically(df: pl.DataFrame) -> pl.DataFrame:
    # Get the list of column names and sort them alphabetically
    sorted_columns = sorted(df.columns)
    
    # Use select to reorder the columns
    return df.select(sorted_columns)

taxonomyResult_mini = capitalize_column_names(taxonomyResult_mini)
taxonomyResult_mini = order_columns_alphabetically(taxonomyResult_mini)

In [11]:
ictv_vmr = pl.read_csv("ictv.csv")
ictv_vmr_mini = ictv_vmr.select(pl.col("Realm,Kingdom,Subkingdom,Phylum,Subphylum,Class,Subclass,Order,Suborder,Family,Subfamily,Genus,Subgenus,Species".split(','))).unique()


In [12]:
pivoted = taxonomyResult_mini.pivot(
    values=[ 'Taxname'],
    index='Sequenceid',
    on='Rank'
)
# pivoted.rename({"no_rank":"Realm"})
pivoted["no rank"].unique()

pivoted = order_columns_alphabetically(capitalize_column_names(pivoted))

In [18]:
final=[]
column_order=["SequenceID", set(ictv_vmr_mini.columns).intersection(pivoted.columns)]
for taxrank in ictv_vmr_mini.columns:
    if taxrank in pivoted.columns:
        pivoted_taxrank_only = pivoted.select([pl.col(taxrank),"Sequenceid"] )
        pivoted_taxrank_only = pivoted_taxrank_only.drop([col for col in pivoted_taxrank_only.columns if col not in [taxrank,"Sequenceid","Score"] ])
        pivoted_taxrank_only = pivoted_taxrank_only.join(ictv_vmr_mini, how="left", on=taxrank)
        pivoted_taxrank_only = order_columns_alphabetically(pivoted_taxrank_only)
        final.append(pivoted_taxrank_only)
final = pl.concat(final, how="vertical").unique()
final = final.unique(subset=["Sequenceid"])

In [19]:
tiny = taxonomyResult_mini.select(["Sequenceid","Score"])
final = tiny.join(final, how="left", on="Sequenceid")
# duplicate the score column for each rank automatically
for taxrank in ictv_vmr_mini.columns:
    if taxrank in final.columns:
        final = final.with_columns(pl.col("Score").alias(f"{taxrank}_score"))

# clean up the output, add fasta headers of contig names not in the mmseqs output
final = final.rename({"Sequenceid":"SequenceID"}) 
final = final.drop(["Score"])


In [20]:
input_fasta = "ictv_chellenge.fasta"
contig_names = {}
with open(input_fasta, 'r') as fasta_file:
    for line in fasta_file:
        if line.startswith('>'):
            contig_name = line.strip()[1:]
            contig_names[contig_name] = True
# add the missing contig names to the final dataframe,
missing_contigs = pl.DataFrame({"SequenceID": list(set(contig_names.keys()) - set(final["SequenceID"]))})
# set scores of all tanks to 1, and set all rank to "unclassified"
for taxrank in ictv_vmr_mini.columns:
    if taxrank in final.columns:
        missing_contigs = missing_contigs.with_columns(pl.lit("unclassified").alias(taxrank))
        # set scores of all tanks to 1
        missing_contigs = missing_contigs.with_columns(pl.lit(1.0).alias(f"{taxrank}_score"))


In [58]:
final2 = pl.concat([order_columns_alphabetically(final), order_columns_alphabetically(missing_contigs)])
example_submission = pl.read_csv("/clusterfs/jgi/scratch/science/metagen/neri/code/blits/ictv-mmseqs2-protein-database/classification_template.csv", separator=",")
missing_columns = [col for col in example_submission.columns if col not in final2.columns]
rename_dict = {col.split(" ")[0]:col for col in missing_columns}
missing_columns2 = [col for col in rename_dict.keys() if col not in final2.columns]
notmissing_columns = [col for col in rename_dict.keys() if col in final2.columns]
rename_dict = {k:v for k,v in rename_dict.items() if k in notmissing_columns}
# rename_dict
final2 = final2.rename(rename_dict)
for col in missing_columns2:
    if col.count("_score") > 0:
        final2 = final2.with_columns(pl.lit(1.0).alias(col))
    else:
        col_wsuffix = [col2 for col2 in example_submission.columns if col2.count(col) != 0][0]
        final2 = final2.with_columns(pl.lit("unclassified").alias(col_wsuffix))

final2 = final2.select(example_submission.columns)
final2.write_csv("submission_ctcv.csv", separator=",",null_value="")